In [1]:
import pandas as pd
from mmrotate.structures.bbox import rbox2qbox
import torch
import PIL.ImageDraw as ImageDraw
import PIL.Image as Image
import os
from tqdm import tqdm

/home/jose/miniconda3/envs/OCD/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PATH_BBOXES_DF = '/home/jose/Programas/OCDDataset/detection_outputs/consolidated_df.csv'
BEST_DETECTOR = 'oriented-reppoints-qbox_r50_fpn_1x_ocd-cropped'
GROUND_TRUTH = 'ground-truth'
PATH_OUTPUT = '/home/jose/Programas/OCDDataset/detection_outputs/best_detector_comparison'
PATH_IMAGES = '/home/jose/Programas/OCDDataset/mmrotate-1.x/data/OCD/test/images'

In [3]:
bboxes_df = pd.read_csv(PATH_BBOXES_DF)
bboxes_df = bboxes_df.loc[bboxes_df['evaluator'].isin([BEST_DETECTOR, GROUND_TRUTH])]
assert len(bboxes_df[bboxes_df['evaluator'] == BEST_DETECTOR]) > 0
assert len(bboxes_df[bboxes_df['evaluator'] == GROUND_TRUTH]) > 0

bboxes_df

,file_name,evaluator,cx,cy,width,height,angle,detection_threshold,type
0,exp_M4_Mauricio_mcr5_frame_40,ground-truth,29.50000,247.00000,96.000000,59.000000,1.570796,1.000000,test
1,exp_M4_Mauricio_mcr5_frame_40,ground-truth,104.39654,581.25867,122.377594,83.773506,1.165904,1.000000,test
2,exp_M4_Mauricio_mcr5_frame_40,ground-truth,196.55138,641.21810,88.093500,149.375370,1.098167,1.000000,test
3,exp_M4_Mauricio_mcr5_frame_40,ground-truth,399.00006,710.00006,143.676730,108.300520,0.501013,1.000000,test
4,exp_M4_Mauricio_mcr5_frame_40,ground-truth,233.97860,826.72015,88.457920,144.678960,0.096918,1.000000,test
...,...,...,...,...,...,...,...,...,...
12623,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,oriented-reppoints-qbox_r50_fpn_1x_ocd-cropped,1181.93270,213.15894,52.744617,41.818580,1.001405,0.051662,test
12624,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,oriented-reppoints-qbox_r50_fpn_1x_ocd-cropped,593.69037,575.59880,19.525280,31.755392,0.155508,0.051504,test
12625,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,oriented-reppoints-qbox_r50_fpn_1x_ocd-cropped,804.45087,462.23666,49.274690,14.411232,1.071601,0.051383,test
12626,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,oriented-reppoints-qbox_r50_fpn_1x_ocd-cropped,746.05860,201.34624,49.128330,82.680940,1.016328,0.050750,test


In [4]:
def getBBoxesFromDF(df, evaluator: str):
    return rbox2qbox(
        torch.Tensor(
            df.loc[(df['evaluator'] == evaluator) & (df['detection_threshold'] >= 0.5)][['cx','cy','width','height','angle']].values)
    )


for image_name, image_group in tqdm(bboxes_df.groupby('file_name')):
    path_input_image = os.path.join(PATH_IMAGES, image_name + '.png')
    path_output_image = os.path.join(PATH_OUTPUT, image_name + '_output.png')

    gt_bboxes = getBBoxesFromDF(image_group, GROUND_TRUTH)
    pred_bboxes = getBBoxesFromDF(image_group, BEST_DETECTOR)

    img = Image.open(path_input_image)
    draw = ImageDraw.Draw(img)
    
    for bbox in gt_bboxes:
        draw.polygon(bbox.tolist(), outline="green", width=2)
    for bbox in pred_bboxes:
        draw.polygon(bbox.tolist(), outline="blue", width=2)
    
    img.save(path_output_image)



100%|██████████| 30/30 [00:10<00:00,  2.99it/s]
